In [ ]:
api_key = 'your_api_key'

tournament_id = 'tournament_id'
headers = {'accept': 'application/vnd.api+json',
           'Authorization': f'Bearer {api_key}'}

tournament_url = 'https://api.pubg.com/tournaments/{tournament_id}'
tournament_response = requests.get(tournament_url, headers=headers)
tournament_data = tournament_response.json()


match_ids = [match['id'] for match in tournament_data['data']['relationships']['matches']['data']]

print(f"Found {len(match_ids)} matches in the tournament")

kill_logs = []

for match_id in match_ids:
    match_url = f'https://api.pubg.com/shards/tournament/matches/{match_id}'
    match_response = requests.get(match_url, headers=headers)
    match_data = match_response.json()

    if 'included' not in match_data:
        print(f"No included data for match {match_id}")
        continue
    
    print(f"Processing match {match_id}")

    if 'assets' in match_data['data']['relationships']:
        asset_id = match_data['data']['relationships']['assets']['data'][0]['id']
    else:
        print(f"No assets data found for match {match_id}")
        continue
    
    telemetry_url = None
    for asset in match_data['included']:
        if asset['type'] == 'asset' and asset['id'] == asset_id:
            telemetry_url = asset['attributes']['URL']
            break

    if not telemetry_url:
        print(f"No telemetry data found for match {match_id}")
        continue

    telemetry_response = requests.get(telemetry_url)
    telemetry_data = telemetry_response.json()


    for event in telemetry_data:
        if event['_T'] == 'LogPlayerKill':
            if event.get('killer') and event['killer'].get('name') and event['killer'].get('location') and event.get('victim') and event['victim'].get('location'):
                killer_name = event['killer']['name']
                killer_location = (event['killer']['location']['x'], event['killer']['location']['y'])
                victim_name = event['victim']['name']
                victim_location = (event['victim']['location']['x'], event['victim']['location']['y'])
                time = event['_D']
                map_name = event['mapName'] if 'mapName' in event else match_data['data']['attributes']['mapName']
                # 좌표 유효성 검사
                if 0 <= killer_location[0] <= 800000 and 0 <= killer_location[1] <= 800000 and 0 <= victim_location[0] <= 800000 and 0 <= victim_location[1] <= 800000:
                    kill_logs.append({
                        'killer': killer_name,
                        'killer_location_x': killer_location[0],
                        'killer_location_y': killer_location[1],
                        'victim': victim_name,
                        'victim_location_x': victim_location[0],
                        'victim_location_y': victim_location[1],
                        'time': time,
                        'match_id': match_id,
                        'map_name': map_name
                    })
                    print(f"Found kill event: {killer_name} killed {victim_name} at {killer_location} on {time}")
                else:
                    print(f"Invalid location: {killer_location} or {victim_location} for match {match_id}")
            else:
                print(f"Incomplete kill event data in match {match_id}")

if kill_logs:
    print(f"Total kill events found: {len(kill_logs)}")
    for log in kill_logs:
        print(log)
else:
    print("No kill events found")


kill_logs_df = pd.DataFrame(kill_logs)


map_images = {
    'Erangel_Main': './Erangel.jpg',
    'Desert_Main': './Miramar.jpg',
    'Savage_Main': './Sanhok.jpg',
}

for map_name, image_path in map_images.items():
    img = mpimg.imread(image_path)
    fig, ax = plt.subplots()
    ax.imshow(img, extent=[0, 800000, 0, 800000])

    # 해당 맵의 킬 로그 표시
    for log in kill_logs:
        if log['map_name'] == map_name:
            killer_x, killer_y = log['killer_location_x'], log['killer_location_y']
            victim_x, victim_y = log['victim_location_x'], log['victim_location_y']
            ax.plot(killer_x, killer_y, 'ro')  # 'ro'는 빨간색 점을 의미 (킬러 위치)
            ax.plot(victim_x, victim_y, 'bo')  # 'bo'는 파란색 점을 의미 (희생자 위치)

    ax.set_title(f"Kill and Death Locations on {map_name}")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.show()